# Looking for associations with microbiome-derived metrics

Okay, here is the pitch: suppose you have a novel metric/measure derived on microbiome data and you already calculated this measure for the amplicon sequencing samples in Arivale. What you want to do know is to find feature in the Arivale multiomics data that are associated/correlated with this new measure. This notebook will show you how to do that in a decently efficient manner.

For the purpose of having an example we will use alpha diversity as an example. That's really not a novel measure but is one we already have. Let's start by getting the alpha diversity from the Arivale data.

In [5]:
import warnings
from arivale_data_interface import get_snapshot

warnings.simplefilter("ignore")

div = get_snapshot("microbiome_diversity")
div.head()

,public_client_id,vendor_observation_id,vendor_dashboard,client_id,vendor_research,days_in_program,days_since_first_call,days_since_first_draw,month,weekday,...,observed_otus_30000,observed_otus_50000,shannon_100000,shannon_20000,shannon_30000,shannon_50000,simpson_100000,simpson_20000,simpson_30000,simpson_50000
0,01000261,AV15-1592,Second Genome,1966.0,amagis,77.0,3.0,12.0,Jan,Sun,...,930.0,1133.9,4.287808,4.258003,4.264935,4.280184,0.957,0.957,0.957,0.957
1,01001298,22001805511243,research-microbiome,275178.0,amagis,16.0,9.0,9.0,Sep,Thu,...,494.3,596.4,4.006391,3.986289,3.993914,4.000846,0.954,0.954,0.954,0.954
2,01001621,22001612562075,research-microbiome,185773.0,amagis,186.0,144.0,175.0,Jan,Mon,...,453.8,545.2,3.849739,3.832411,3.844887,3.847660,0.955,0.955,0.955,0.955
3,01001661,22001612561628,research-microbiome,177541.0,amagis,20.0,6.0,-4.0,Jun,Mon,...,453.9,560.6,NaN,3.732598,3.736063,3.740222,NaN,0.937,0.937,0.937
4,01001798,22001805510925,research-microbiome,278782.0,amagis,12.0,7.0,9.0,Jan,Mon,...,581.1,720.5,3.835876,3.814389,3.826866,3.833797,0.951,0.951,0.951,0.951


The minimum information you will need to keep around is the measure, the public_client_id, vendor_observation_id and days_in_program. If your data only has the vendor_observation_id than you can merge with the the columns in this table to get those other ones. So let's assemble our primary table.

In [2]:
measure = div[["diversity_shannon", "vendor_observation_id", "public_client_id", "days_in_program", "vendor_dashboard"]]

Okay that is what we start with. Let's continue.

## Adding in the confounders

The next thing is the confounders, e.g. covariates we want to correct for since they may be associated with our measure and the omics data without having a direct measure <-> omics data connection. For the microbiome data the principal ones are sex, age and BMI, but you could also correct for additional ones like genetic ancestry for instance. We find those measures in the `clients` and `weights` table and will add those in as well. For the weights we encounter a common challenge. There are multiple measurement points for each individual, so which one do we use for the BMI? We will use the closest data point to the microbiome sample. That is why we kept the `days_in_program` data around.

In [4]:
import pandas as pd

clients = get_snapshot("clients")[["public_client_id", "sex", "age"]]
measure = pd.merge(measure, clients, on="public_client_id", how="inner")

weights = get_snapshot("weight")[["BMI_CALC", "public_client_id", "days_in_program"]].dropna()
measure = pd.merge_asof(
    measure[measure.public_client_id.isin(weights.public_client_id)].sort_values(by="days_in_program"), 
    weights.sort_values(by="days_in_program"), 
    by="public_client_id", on="days_in_program", direction="nearest")

measure.shape

NameError: name 'measure' is not defined

In [4]:
measure.head()

,diversity_shannon,vendor_observation_id,public_client_id,days_in_program,vendor_dashboard,sex,age,BMI_CALC
0,3.870313,AV15-4372,01602320,-2.0,Second Genome,M,48.0,37.229730
1,4.281822,AV15-3716,01228476,0.0,Second Genome,F,51.0,22.462722
2,3.318960,AV15-3588,01949838,1.0,Second Genome,F,32.0,25.328546
3,3.683513,AV15-3840,01602518,1.0,Second Genome,F,50.0,34.154541
4,4.009163,22001612562257,01360278,1.0,research-microbiome,F,43.0,20.938965


Okay, some individuals were dropped due to missing data. No biggie. But at least we are now ready to actually start with our association analysis. We will start with the metabolomics data to illustrate the steps for the common omics analyses.

## Metabolome

Okay each of those analysis will essentially use the same steps which are:
1. identify metadata and actual data columns
2. merge the omics data with the measure
3. transform or normalize and filter features
4. go through each feature and merge it with the metadata
5. run the regression for each feature and track the results
6. merge the results with the feature annotations

Let's go then.

### 1. Identify metadata and actual data columns

We do this so we know which columns denote metadata and which ones actual measures (metabolite abundances).

In [3]:
metabolites = get_snapshot("metabolomics_corrected").sort_values(by="days_in_program")
measure = measure.sort_values(by="days_in_program")
metabolite_features = metabolites.columns[8:]
metabolites.days_in_program = metabolites.days_in_program.astype("float64")

NameError: name 'measure' is not defined

In [6]:
metabolite_features

Index(['35', '50', '55', '62', '71', '93', '98', '111', '112', '117',
       ...
       '999953114', '999953157', '999953172', '999953266', '999953267',
       '999954831', '999954832', '999954834', '999954839', '999954840'],
      dtype='object', length=1296)

In [7]:
metabolites.columns

Index(['public_client_id', 'sample_id', 'days_in_program',
       'days_since_first_call', 'days_since_first_draw', 'month', 'weekday',
       'season', '35', '50',
       ...
       '999953114', '999953157', '999953172', '999953266', '999953267',
       '999954831', '999954832', '999954834', '999954839', '999954840'],
      dtype='object', length=1304)

### 2. Merge metadata and measure

In [8]:
metabolites_merged = pd.merge_asof(
    measure, metabolites, 
    by="public_client_id", 
    on="days_in_program", 
    direction="nearest", 
    tolerance=30.0).dropna(subset=metabolite_features, how="all")
metabolite_features = metabolites.columns[metabolites.columns.isin(metabolite_features)]
metabolites_merged

,diversity_shannon,vendor_observation_id,public_client_id,days_in_program,vendor_dashboard,sex,age,BMI_CALC,sample_id,days_since_first_call,...,999953114,999953157,999953172,999953266,999953267,999954831,999954832,999954834,999954839,999954840
3,4.278019,AV15-1348,01306924,1.0,Second Genome,M,56.0,26.541837,A422AU094-002,-155.0,...,1.528782,1.606430,1.125981,0.896099,0.838319,1.023423,1.075749,1.051521,0.766951,1.026390
6,4.283908,AV15-1295,01421882,1.0,Second Genome,F,19.0,23.793846,A752AT612-002,-5.0,...,1.147181,0.758275,1.814588,3.996297,0.935515,0.915287,0.867081,0.871120,1.224561,NaN
7,4.403993,AV15-1094,01789546,1.0,Second Genome,F,87.0,18.692937,A976AR221-002,-6.0,...,1.153663,0.658271,0.338122,1.162175,1.845431,0.963192,0.942735,0.961368,0.650608,1.703106
8,4.542282,AV15-3636,01447446,1.0,Second Genome,F,56.0,21.950816,A904AZ485-002,1.0,...,NaN,NaN,NaN,1.522402,1.073743,1.063270,1.146081,0.942299,1.696602,1.564450
9,4.009163,22001612562257,01360278,1.0,research-microbiome,F,43.0,20.938965,A089BI756-002,24.0,...,NaN,1.260702,NaN,NaN,1.391536,NaN,NaN,NaN,NaN,1.055905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4468,3.684504,AV15-2674,01801678,453.0,Second Genome,M,35.0,20.705900,A706BA371-002,384.0,...,0.514679,NaN,0.768327,1.997009,1.036581,NaN,NaN,NaN,NaN,0.570969
4486,2.850451,AV15-2217,01874590,463.0,Second Genome,M,36.0,21.558261,A928BB321-002,384.0,...,NaN,NaN,1.382292,0.963556,1.770288,NaN,NaN,NaN,NaN,0.829983
4551,3.914202,22001805511808,01941857,509.0,research-microbiome,M,54.0,22.148708,A143BO102-003,488.0,...,NaN,1.016756,NaN,1.035061,0.566952,NaN,NaN,NaN,1.032233,1.101631
4564,4.352964,22001612561459,01524770,517.0,research-microbiome,F,53.0,21.797160,A404BO077-008,531.0,...,1.649832,1.048798,3.931804,1.391538,1.358102,NaN,NaN,NaN,NaN,0.490642


Okay we reduced the data set quite by a bit because we requested that we only want samples where blood and fecal samples were talken within 30 days. You can adjust this cutoof however you see fit. 

### 3. Preprocess data

This depends a bit on your input but we usually wan to transform our features in a ways that they are comparable and not too rare. So here let's consider only metabolites that appear in at least 75% of all samples. The values are also all positive so we will log-transform and standardize them.

In [9]:
bad = metabolites_merged[metabolite_features].isnull().sum() / metabolites_merged.shape[0] > 0.25
metabolite_features = bad[~bad].index
metabolites_merged = metabolites_merged.drop(columns=bad[bad].index)
metabolites_merged.shape

(1778, 929)

In [10]:
import numpy as np

metabolites_merged[metabolite_features] = np.log(metabolites_merged[metabolite_features])
metabolites_merged[metabolite_features] = metabolites_merged[metabolite_features].apply(lambda x: (x - x.mean()) / x.std())
metabolites_merged.rename(columns=dict(zip(metabolite_features, "metabolite_" + metabolite_features)), inplace=True)
metabolite_features = "metabolite_" + metabolite_features

In [11]:
assert (metabolites_merged[metabolite_features].mean().abs() < 1e-6).all()

In [12]:
metabolites_merged.head()

,diversity_shannon,vendor_observation_id,public_client_id,days_in_program,vendor_dashboard,sex,age,BMI_CALC,sample_id,days_since_first_call,...,metabolite_999952524,metabolite_999952533,metabolite_999952640,metabolite_999952843,metabolite_999952865,metabolite_999952867,metabolite_999952877,metabolite_999952909,metabolite_999953157,metabolite_999953267
3,4.278019,AV15-1348,01306924,1.0,Second Genome,M,56.0,26.541837,A422AU094-002,-155.0,...,-1.068655,0.677474,-0.560102,-1.702583,1.092412,0.495039,0.654090,0.672189,1.055484,-0.466214
6,4.283908,AV15-1295,01421882,1.0,Second Genome,F,19.0,23.793846,A752AT612-002,-5.0,...,-0.924359,0.402097,0.146529,NaN,0.412785,-1.952615,NaN,0.114193,-0.565776,-0.216730
7,4.403993,AV15-1094,01789546,1.0,Second Genome,F,87.0,18.692937,A976AR221-002,-6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.629504,-0.871208,1.328348
8,4.542282,AV15-3636,01447446,1.0,Second Genome,F,56.0,21.950816,A904AZ485-002,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.096684
9,4.009163,22001612562257,01360278,1.0,research-microbiome,F,43.0,20.938965,A089BI756-002,24.0,...,0.144189,NaN,-0.069903,NaN,0.315495,-0.467529,0.837372,0.282680,0.532115,0.686309


### 4. Run the regressions

Now we have a clean data set and can start to run the regressions. You will see that this will surprisingly make use of all cores of the machine. This is because conda is setup by default to parallelize basic numeric operations like matrix factorization. Neat!

In [13]:
from rich.progress import track
from statsmodels.formula.api import ols
from statsmodels.stats.multitest import multipletests

def get_results(feature):
    """Get a single association.
    
    As long as `args` and `df` are assigned this can be used on
    any data set.
    """
    formula = f"{feature} ~ C(sex) + age + BMI_CALC + C(vendor_dashboard) + diversity_shannon"
    fitted = ols(formula, data=df).fit()
    return pd.DataFrame({
        "feature": feature,
        "beta": fitted.params["diversity_shannon"],
        "t_statistic": fitted.tvalues["diversity_shannon"],
        "p": fitted.pvalues["diversity_shannon"],
        "n": fitted.nobs
        }, index=[feature])

args = metabolite_features
df = metabolites_merged
results = map(get_results, track(args))
results = list(results)
tests = pd.concat(results)
tests["q"] = multipletests(tests.p, method="fdr_bh")[1]

### 5. Merge in annotations

Finally we merge in the metadata for the metabolites.

In [14]:
anns = get_snapshot("metabolomics_metadata").iloc[:, 0:8]
anns["feature"] = "metabolite_" + anns.CHEMICAL_ID.astype(str)
tests = pd.merge(tests, anns, on="feature")

In [15]:
# tests.to_csv("metabolome_tests.csv", index=False)  # uncomment to save the result
tests.sort_values(by="q").head(10)

,feature,beta,t_statistic,p,n,q,CHEMICAL_ID,SUB_PATHWAY,SUPER_PATHWAY,BIOCHEMICAL_NAME,CAS,KEGG,HMDB,PUBCHEM
420,metabolite_100002253,1.075114,20.635567,6.062476e-84,1583.0,5.547166e-81,100002253,Food Component/Plant,Xenobiotics,cinnamoylglycine,16534-24-0,NaN,HMDB11621,709625.0
167,metabolite_100000010,1.051025,19.046454,8.262365e-73,1549.0,3.780032e-70,100000010,Phenylalanine and Tyrosine Metabolism,Amino Acid,3-phenylpropionate (hydrocinnamate),501-52-0,C05629,HMDB00764,107.0
425,metabolite_100002488,-0.977719,-18.893243,3.033731e-72,1660.0,9.252880e-70,100002488,Secondary Bile Acid Metabolism,Lipid,isoursodeoxycholate,78919-26-3,C17662,HMDB00686,127601.0
828,metabolite_999946613,0.975420,18.852741,5.855392e-72,1657.0,1.339421e-69,999946613,NaN,NaN,X - 12216,NaN,NaN,NaN,NaN
283,metabolite_100001315,0.814344,15.440487,2.029910e-50,1698.0,3.714736e-48,100001315,Phenylalanine and Tyrosine Metabolism,Amino Acid,p-cresol sulfate,3233-57-7,C01468,HMDB11635,4615423.0
168,metabolite_100000014,0.792415,15.171555,8.639278e-49,1668.0,1.317490e-46,100000014,Benzoate Metabolism,Xenobiotics,hippurate,495-69-2,C01586,HMDB00714,464.0
395,metabolite_100002021,0.794967,14.664319,1.722709e-45,1449.0,2.251826e-43,100002021,Steroid,Lipid,"5alpha-androstan-3beta,17alpha-diol disulfate",NaN,NaN,NaN,NaN
812,metabolite_999946507,0.880734,13.685530,3.984056e-40,1418.0,4.556764e-38,999946507,NaN,NaN,X - 11850,NaN,NaN,NaN,NaN
355,metabolite_100001657,0.699066,12.757053,1.226240e-35,1680.0,1.246678e-33,100001657,Secondary Bile Acid Metabolism,Lipid,glycolithocholate sulfate*,15324-64-8,C11301,HMDB02639,72222.0
438,metabolite_100002911,-0.697544,-12.524210,1.882490e-34,1666.0,1.722478e-32,100002911,Secondary Bile Acid Metabolism,Lipid,glycoursodeoxycholate,64480-66-6,NaN,HMDB00708,12310288.0


Okay, that was this. You could do exactly the same for proteomics. Let's run through it with clinical labs here since we don't really want to adjust the variables that much. We will group the steps this time.

## Clinical labs

In [16]:
# Find the features
chems = get_snapshot("chemistries", clean=True).sort_values(by="days_in_program")
chem_features = chems.columns[12:]  # inspect the table to find out where the metadata ends 
chems.days_in_program = chems.days_in_program.astype("float64")

# Merge with measure
chems_merged = pd.merge_asof(
    measure, chems, 
    by="public_client_id", 
    on="days_in_program", 
    direction="nearest", 
    tolerance=30.0).dropna(subset=chem_features, how="all")
chem_features = chems.columns[chems.columns.isin(chem_features)]

# Filter uncomplete
bad = chems_merged[chem_features].isnull().sum() / chems_merged.shape[0] > 0.25
chem_features = bad[~bad].index
chems_merged = chems_merged.drop(columns=bad[bad].index)

# We don't transform the values further since some of them are integer etc. and they have associated units we don't want to break
# However, that mean we can't compare the beta values between features.

In [18]:
chems_merged.columns

Index(['diversity_shannon', 'vendor_observation_id_x', 'public_client_id',
       'days_in_program', 'vendor_dashboard', 'sex', 'age', 'BMI_CALC',
       'vendor', 'vendor_observation_id_y', 'observation_id', 'reflexive',
       'fasting', 'days_since_first_call', 'days_since_first_draw', 'month',
       'weekday', 'season', 'A/G RATIO', 'ALAT (SGPT)', 'ALBUMIN',
       'ALKALINE PHOSPHATASE', 'ARACHIDONIC ACID', 'ASAT (SGOT)', 'BASOPHILS',
       'BASOPHILS ABSOLUTE', 'BILIRUBIN, TOTAL', 'BUN/CREAT RATIO', 'CALCIUM',
       'CARBON DIOXIDE (CO2)', 'CHLORIDE', 'CHOLESTEROL, TOTAL',
       'CREATININE ENZ, SER', 'CRP HIGH SENSITIVITY', 'DHA', 'DPA',
       'EOSINOPHILS', 'EOSINOPHILS ABSOLUTE', 'EPA', 'FERRITIN', 'GFR, MDRD',
       'GFR, MDRD, AFRICAN AM', 'GGT', 'GLOBULIN', 'GLUCOSE',
       'GLYCOHEMOGLOBIN A1C', 'HDL CHOL DIRECT', 'HDL PARTICLE NUMBER',
       'HEMATOCRIT', 'HEMOGLOBIN', 'HOMA-IR', 'HOMOCYSTEINE, SERUM',
       'IMMATURE GRANULOCYTES', 'IMMATURE GRANULOCYTES ABSOLUT

Looks pretty concise this way and we can start to run the testing loop again.

In [17]:
args = chem_features
df = chems_merged  # don't forget to reassign those
results = map(get_results, track(args))
results = list(results)
tests_chems = pd.concat(results)
tests_chems["q"] = multipletests(tests_chems.p, method="fdr_bh")[1]

SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
anns = get_snapshot("chemistries_metadata")
anns["feature"] = anns.Name
tests_chems = pd.merge(tests_chems, anns, on="feature")

In [ ]:
tests_chems.sort_values(by="q").head(10)

## Questionnaires

Okay same things as before but we get the questionnaire data from the general assessments and will express it on an integer scale from 0-M where 0 is little and M is a lot. For that we will only use the "enum" and "int" scales.

In [19]:
qs = get_snapshot("assessments", clean=True).sort_values(by="days_in_program")
q_features = qs.columns[8:]
q_features = q_features[q_features.str.endswith("_enum") | q_features.str.endswith("_int")]
qs.days_in_program = qs.days_in_program.astype("float64")

# Merge with measure
qs_merged = pd.merge_asof(
    measure, qs.dropna(subset="days_in_program"), 
    by="public_client_id", 
    on="days_in_program", 
    direction="nearest", 
    tolerance=30.0).dropna(subset=q_features, how="all")
q_features = qs.columns[qs.columns.isin(q_features)]

# Filter uncomplete
bad = qs_merged[q_features].isnull().sum() / qs_merged.shape[0] > 0.25
q_features = bad[~bad].index
qs_merged = qs_merged.drop(columns=bad[bad].index)

# The data is all categorical so we will make it ordinal now
qs_merged[q_features] = qs_merged[q_features].apply(lambda x: x.str.split("\\(|\\)", regex=True).str[1]).astype("float64")
qs_merged

,diversity_shannon,vendor_observation_id,public_client_id,days_in_program,vendor_dashboard,sex,age,BMI_CALC,vendor,days_since_first_call,...,assessment:pss-four-item:insurmountable:enum,assessment:pss-four-item:out_of_control:enum,assessment:vitals:antibiotics:enum,assessment:vitals:blood_pressure:enum,assessment:vitals:blood_sugar:enum,assessment:vitals:cholesterol:enum,assessment:vitals:colonoscopy:enum,assessment:vitals:ethnicity:enum,assessment:vitals:mammogram:enum,assessment:vitals:race:enum
45,2.712285,22001805511579,01156594,3.0,research-microbiome,F,72.0,32.220833,Assessments,-9.0,...,(1) Almost Never,(1) Almost Never,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,4.530410,22001803513494,01362239,4.0,research-microbiome,F,70.0,22.594123,Assessments,-4.0,...,(0) Never,(0) Never,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,3.257099,22001805511823,01730310,4.0,research-microbiome,M,51.0,28.693878,Assessments,0.0,...,(3) Fairly Often,(1) Almost Never,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,3.562083,22001805520631,01712678,4.0,research-microbiome,F,32.0,42.252038,Assessments,-13.0,...,(2) Sometimes,(2) Sometimes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,3.620308,22001804570150,01225920,4.0,research-microbiome,M,40.0,34.256669,Assessments,-21.0,...,(2) Sometimes,(3) Fairly Often,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5071,3.362457,22001805521425,01994330,1348.0,research-microbiome,M,48.0,28.112085,Assessments,1309.0,...,(1) Almost Never,(1) Almost Never,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5074,3.354139,22001805510143,01108982,1354.0,research-microbiome,M,62.0,22.081310,Assessments,1311.0,...,(1) Almost Never,(0) Never,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5076,3.868454,22001805521073,01441946,1367.0,research-microbiome,F,44.0,19.732234,Assessments,1334.0,...,(2) Sometimes,(2) Sometimes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5078,4.207403,22001805510387,01023946,1371.0,research-microbiome,F,43.0,27.259184,Assessments,1339.0,...,(2) Sometimes,(3) Fairly Often,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


And we test again.

In [20]:
args = q_features
df = qs_merged  # don't forget to reassign those
results = map(get_results, track(args))
results = list(results)
tests_qs = pd.concat(results)
tests_qs["q"] = multipletests(tests_qs.p, method="fdr_bh")[1]

For the annotations we will simply append the meaning of the numeric scale here. Makes it easier to interpret the result (like the directon of beta).

In [21]:
scales = qs[q_features].apply(lambda x: " | ".join(x.astype(str).sort_values().unique()) if x.dtype == "O" else "numeric")
tests_qs["scales"] = scales[tests_qs.feature].values

In [22]:
tests_qs.sort_values(by="q").head(10)

,feature,beta,t_statistic,p,n,q,scales
assessment_health_history_prescription_medication_use_yes_no_int,assessment_health_history_prescription_medicat...,-0.087469,-2.585243,0.009878,964.0,0.079021,(1) No | (2) Yes | nan
assessment_health_history_drugs_asthma_int,assessment_health_history_drugs_asthma_int,-0.080982,-1.932916,0.053542,961.0,0.214166,(0) Never | (1) Currently | (2) Formerly | nan
assessment_health_history_drugs_allergy_int,assessment_health_history_drugs_allergy_int,-0.061368,-1.214417,0.224889,961.0,0.599703,(0) Never | (1) Currently | (2) Formerly | nan
assessment_health_history_drugs_anti_coagulation_int,assessment_health_history_drugs_anti_coagulati...,-0.005262,-0.176631,0.859836,960.0,0.859836,(0) Never | (1) Currently | (2) Formerly | nan
assessment_health_history_drugs_osteoporosis_int,assessment_health_history_drugs_osteoporosis_int,-0.004209,-0.215083,0.829749,961.0,0.859836,(0) Never | (1) Currently | (2) Formerly | nan
assessment_health_history_drugs_psychiatric_int,assessment_health_history_drugs_psychiatric_int,-0.028410,-0.657168,0.511231,961.0,0.859836,(0) Never | (1) Currently | (2) Formerly | nan
assessment_health_history_over_the_counter_medication_use_int,assessment_health_history_over_the_counter_med...,0.024875,0.486631,0.626632,957.0,0.859836,(0) No | (1) Rarely/Occasionally | (2) Yes | nan
assessment_health_history_supplement_use_int,assessment_health_history_supplement_use_int,-0.024872,-0.447417,0.654676,957.0,0.859836,(0) No | (1) Rarely/Occasionally | (2) Yes | nan
